In [20]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [21]:
pred_cc_img_path = '/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc' 
#image_100_cc.png
lbl_cc_img_path = '/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc' #
# image_100_gt_cc.tif

### Common Function

In [22]:
range_found = 'no range found'
def find_iou_range(iou_score):
    if 0 <= iou_score <= 20:
        range_found = ' <= 20%'
    elif 20 <= iou_score <= 30:
        range_found = '20+ to 30%'
    elif 30 <= iou_score <= 40:
        range_found = '30+ to 40%'
    elif 40 <= iou_score <= 50:
        range_found = '40+ to 50%'
    elif 50 <= iou_score <= 60:
        range_found = '50+ to 60%'
    elif 60 <= iou_score <= 70:
        range_found = '60+ to 70%'
    elif 70 <= iou_score <= 80:
        range_found = '70+ to 80%'
    elif 80 <= iou_score <= 90:
        range_found = '80+ to 90%'        
    elif 90 <= iou_score <= 95:
        range_found = '90+ to 95%'        
    elif 95 <= iou_score <= 100:
        range_found = '95+ to 100%'                

    return range_found

In [23]:
def find_iou(gt_bw_img, pred_bw_img):
    if(gt_bw_img.shape == pred_bw_img.shape):
        intersection = np.logical_and(gt_bw_img, pred_bw_img)
        union = np.logical_or(gt_bw_img, pred_bw_img)
        iou_score = np.sum(intersection) / np.sum(union)
        return iou_score
    else:
        raise Exception('***** find_iou() fails.')

### Read Images

In [24]:
lst_pred_cc_imgs = [img for img in os.listdir(pred_cc_img_path) if os.path.isfile(os.path.join(pred_cc_img_path, img))]
lst_pred_cc_imgs[:2], len(lst_pred_cc_imgs)

(['image_541_cc_binary.png', 'image_473_cc_binary.png'], 626)

### Find IOU and Write it to CSV

In [26]:
lst_final_csv_output= []
for idx, pred_img_name in enumerate(lst_pred_cc_imgs, start=1):
    try:
#         print(pred_img_name)
        pred_img_path_with_filename = os.path.join(pred_cc_img_path, pred_img_name)

        lbl_img_name = pred_img_name.replace('_cc_binary.png', '_gt_cc.tif')
        lbl_img_path_with_filename = os.path.join(lbl_cc_img_path, lbl_img_name)
        print(lbl_img_path_with_filename)
        print(pred_img_path_with_filename)

        lbl_bw_img = cv.imread(lbl_img_path_with_filename)
        pred_bw_img = cv.imread(pred_img_path_with_filename)

        #find IOU score
        iou_score_original = find_iou(lbl_bw_img, pred_bw_img)
        iou_score = '{0:.4f}'.format(iou_score_original)

        iou_perc = '{0:.4f}'.format(iou_score_original*100)
    #         print(type(iou_perc), iou_perc)
        iou_range = find_iou_range(float(iou_perc))
        img_name = pred_img_name.replace('cc_binary.png', '')
        print_str = '# {} :\t {} \t {} \t {} \t {} \t {}'.format(idx, lbl_img_name, pred_img_name, iou_score, iou_perc, iou_range)
        print(print_str)
    #         print('Processing Image # %d : %s \t %s \t %s \t %s' % (idx, img_name, pred_file_name, iou_score, iou_perc))
    #     write_current_progress_status(print_str)
        #construct LIST to store it in csv file
        lst_final_csv_output.append([idx, img_name, iou_score , iou_perc, iou_range])
    except Exception as exception:
        print('-' * 50)
        print('Exception raised %s for the Image %s' % (str(exception), img_name))
        print('-' * 50)
#         write_log(str(exception), img_name )
    finally:
        print('~' * 100)

        
#store the csv output
df = pd.DataFrame(lst_final_csv_output, columns=['#', 'image_name', 'IOU_Score', 'IOU_Perc', 'Class'])
df.to_csv('iou_score.csv', index=False)
print('total files processed : ', idx)
print('csv generated successfully')    

/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_541_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_541_cc_binary.png
# 1 :	 image_541_gt_cc.tif 	 image_541_cc_binary.png 	 0.9579 	 95.7913 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_473_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_473_cc_binary.png
# 2 :	 image_473_gt_cc.tif 	 image_473_cc_binary.png 	 0.9629 	 96.2907 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_226_gt_cc.tif
/home/ec2-use

# 48 :	 image_417_gt_cc.tif 	 image_417_cc_binary.png 	 0.9652 	 96.5212 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_162_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_162_cc_binary.png
# 49 :	 image_162_gt_cc.tif 	 image_162_cc_binary.png 	 0.7506 	 75.0560 	 70+ to 80%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_284_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_284_cc_binary.png
# 50 :	 image_284_gt_cc.tif 	 image_284_cc_binary.png 	 0.7660 	 76.6012 	 70+ to 80%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_586_cc_binary.png
# 95 :	 image_586_gt_cc.tif 	 image_586_cc_binary.png 	 0.9673 	 96.7323 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_611_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_611_cc_binary.png
# 96 :	 image_611_gt_cc.tif 	 image_611_cc_binary.png 	 0.8961 	 89.6102 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_310_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_310_cc_binary.png
# 97 :	 

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_598_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_598_cc_binary.png
# 143 :	 image_598_gt_cc.tif 	 image_598_cc_binary.png 	 0.8946 	 89.4577 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_392_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_392_cc_binary.png
# 144 :	 image_392_gt_cc.tif 	 image_392_cc_binary.png 	 0.9639 	 96.3895 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/

# 188 :	 image_237_gt_cc.tif 	 image_237_cc_binary.png 	 0.6596 	 65.9637 	 60+ to 70%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_64_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_64_cc_binary.png
# 189 :	 image_64_gt_cc.tif 	 image_64_cc_binary.png 	 0.8751 	 87.5084 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_434_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_434_cc_binary.png
# 190 :	 image_434_gt_cc.tif 	 image_434_cc_binary.png 	 0.9554 	 95.5419 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

...

# 426 :	 image_360_gt_cc.tif 	 image_360_cc_binary.png 	 0.9301 	 93.0110 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_408_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_408_cc_binary.png
# 427 :	 image_408_gt_cc.tif 	 image_408_cc_binary.png 	 0.9539 	 95.3942 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_40_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_40_cc_binary.png
# 428 :	 image_40_gt_cc.tif 	 image_40_cc_binary.png 	 0.9277 	 92.7743 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# 473 :	 image_224_gt_cc.tif 	 image_224_cc_binary.png 	 0.6337 	 63.3699 	 60+ to 70%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_623_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_623_cc_binary.png
# 474 :	 image_623_gt_cc.tif 	 image_623_cc_binary.png 	 0.7515 	 75.1525 	 70+ to 80%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_575_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_575_cc_binary.png
# 475 :	 image_575_gt_cc.tif 	 image_575_cc_binary.png 	 0.8031 	 80.3143 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# 520 :	 image_480_gt_cc.tif 	 image_480_cc_binary.png 	 0.9667 	 96.6728 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_581_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_581_cc_binary.png
# 521 :	 image_581_gt_cc.tif 	 image_581_cc_binary.png 	 0.9589 	 95.8867 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_402_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_402_cc_binary.png
# 522 :	 image_402_gt_cc.tif 	 image_402_cc_binary.png 	 0.9245 	 92.4492 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# 568 :	 image_445_gt_cc.tif 	 image_445_cc_binary.png 	 0.9545 	 95.4536 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_584_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_584_cc_binary.png
# 569 :	 image_584_gt_cc.tif 	 image_584_cc_binary.png 	 0.9725 	 97.2509 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_558_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_558_cc_binary.png
# 570 :	 image_558_gt_cc.tif 	 image_558_cc_binary.png 	 0.9382 	 93.8196 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# 617 :	 image_499_gt_cc.tif 	 image_499_cc_binary.png 	 0.9396 	 93.9625 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_567_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_567_cc_binary.png
# 618 :	 image_567_gt_cc.tif 	 image_567_cc_binary.png 	 0.9707 	 97.0663 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/label-cc/image_428_gt_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/verification-run-27-jan-20/rn-50-cc/image_428_cc_binary.png
# 619 :	 image_428_gt_cc.tif 	 image_428_cc_binary.png 	 0.9511 	 95.1070 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~